In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

cid = "4acff433ba4c4b1e97a5cacdb6358f52"
secret = "bcd9bc8e6123468085c44fa911a613ad"
username = "klinga88"

client_credentials_manager = SpotifyClientCredentials(cid, secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
playlists = sp.user_playlists(username)
my_playlists = {}
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_owner = playlist["uri"].split(":")[2]
        playlist_name = playlist['name']
        playlist_id = playlist["uri"].split(":")[4]
        if playlist_owner == username:
            my_playlists[playlist_name] = playlist_id
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None
        
print(json.dumps(my_playlists, indent=4))

{
    "fireplace jams": "6mrmqvQnT8AAycxXiFTtmd",
    "High Water 2018": "0A28uckGniY1SlD2x5qcH7",
    "thats my JAM": "7rPCUQipVw2qC2phYkpAMa",
    "The NEW New hype": "65TCucy7fYoLNRXZQ15Cct",
    "High Water Festival - 2017": "2Miih4LaDlWKNhgpXBAXMj",
    "runrunrun": "3J06yaQeWJpVN8jvAi3fmG",
    "Spring's Sprung": "7FwED0aB4bvDLnVJ5JXNM0",
    "Better Christmas Music": "48BEDmvjUnLESgf7W0Hn1G",
    "Fall Tunes": "6FIhI4VffV0ZSpIpb4HmbQ",
    "Chi Town": "7AmDw9GvJLORItNQyDajLq",
    "Glass animals": "2xKhDomk2ysdpePO7XtOhZ",
    "Mmmmm Monday": "5L2qyUKtDPpY4yoHD9kXXl",
    "ITS A PARTY": "7JUQfTLglrSDi3aSh2un94",
    "So Indie": "7uDEagBEvTVV04lL0O2O4m",
    "Bells and Whistles": "04erMgjeXFbT3teg9fcuHO",
    "My Poppy: Rock": "59Wd1Ri3SxWkq8Vy4L3W1Y",
    "Boldy James - Grand Quarters": "1DZq2vmyaCjDwLUigSosEi",
    "Liked from Radio": "2q3ZLd74RuyVlyXIjkxEui",
    "Coney Island": "6HBwzPoJrgOKcOmFVgLikn",
    "White Daisy Passing": "0qvdnjdxbxBJ0ENI1s0bnR"
}


In [11]:
#get tracks in each playlist
songs = []
sp_playlist = sp.user_playlist_tracks(username, playlist_id="2Miih4LaDlWKNhgpXBAXMj")
tracks = sp_playlist['items']
print(json.dumps(tracks, indent=4))

[
    {
        "added_at": "2016-11-19T02:23:17Z",
        "added_by": {
            "external_urls": {
                "spotify": "https://open.spotify.com/user/klinga88"
            },
            "href": "https://api.spotify.com/v1/users/klinga88",
            "id": "klinga88",
            "type": "user",
            "uri": "spotify:user:klinga88"
        },
        "is_local": false,
        "primary_color": null,
        "track": {
            "album": {
                "album_type": "album",
                "artists": [
                    {
                        "external_urls": {
                            "spotify": "https://open.spotify.com/artist/73ncl3RMnT90eLFhZdO2WU"
                        },
                        "href": "https://api.spotify.com/v1/artists/73ncl3RMnT90eLFhZdO2WU",
                        "id": "73ncl3RMnT90eLFhZdO2WU",
                        "name": "The Suffers",
                        "type": "artist",
                        "uri": "spotify:a

In [24]:


print(f"length of playlist: {len(tracks)}")
#get each song, song URI, artist, artist URI, album, album URI, album img url
for i in range(len(tracks)):
    try:
        song_name = tracks[i]["track"]["name"]
        song_uri = tracks[i]["track"]["uri"]
        artist_name = tracks[i]["track"]["artists"][0]["name"]# @TODO Update to get all artists on track not just first 
        artist_uri = tracks[i]["track"]["artists"][0]["uri"]
        album_name = tracks[i]["track"]["album"]["name"]
        album_uri = tracks[i]["track"]["album"]["uri"]
        album_img = tracks[i]["track"]["album"]["images"][1]
        songs.append([song_name,song_uri,artist_name,artist_uri,album_name,album_uri,album_img])
    except Exception as e:
        print(f"Exception on track number {i}, {e}")
print(json.dumps(songs,indent=4))

length of playlist: 56
[
    [
        "Make Some Room",
        "spotify:track:4KQVZf1Bb0lCH0VLTHIOk5",
        "The Suffers",
        "spotify:artist:73ncl3RMnT90eLFhZdO2WU",
        "The Suffers",
        "spotify:album:5sDC2t1dOIN0l6pHWCU4Fb",
        {
            "height": 300,
            "url": "https://i.scdn.co/image/1bcc4d798859f76e34c16fe433a617b285caa409",
            "width": 300
        }
    ],
    [
        "Gwan",
        "spotify:track:5yA5XFK0oCl5VLxo6IxjrZ",
        "The Suffers",
        "spotify:artist:73ncl3RMnT90eLFhZdO2WU",
        "The Suffers",
        "spotify:album:5sDC2t1dOIN0l6pHWCU4Fb",
        {
            "height": 300,
            "url": "https://i.scdn.co/image/1bcc4d798859f76e34c16fe433a617b285caa409",
            "width": 300
        }
    ],
    [
        "Peanuts",
        "spotify:track:40Lk7c3tXzlLHREQIW5sk2",
        "The Suffers",
        "spotify:artist:73ncl3RMnT90eLFhZdO2WU",
        "The Suffers",
        "spotify:album:5sDC2t1dOIN0l6pH

In [20]:
analysis = sp.audio_analysis("09eSdS5RTgyodJt3krr5AC")
print(json.dumps(analysis, indent=4))

{
    "meta": {
        "analyzer_version": "4.0.0",
        "platform": "Linux",
        "detailed_status": "OK",
        "status_code": 0,
        "timestamp": 1531281475,
        "analysis_time": 7.20373,
        "input_process": "libvorbisfile L+R 44100->22050"
    },
    "track": {
        "num_samples": 5106780,
        "duration": 231.6,
        "sample_md5": "",
        "offset_seconds": 0,
        "window_seconds": 0,
        "analysis_sample_rate": 22050,
        "analysis_channels": 1,
        "end_of_fade_in": 0.28549,
        "start_of_fade_out": 228.76299,
        "loudness": -8.369,
        "tempo": 175.712,
        "tempo_confidence": 0.056,
        "time_signature": 3,
        "time_signature_confidence": 0.998,
        "key": 11,
        "key_confidence": 0.0,
        "mode": 0,
        "mode_confidence": 0.226,
        "codestring": "eJxVmtmVIzkMBF2RCbwP_x3biKR6uvdHT2RV8QCBRAJgLXX3e-b8lE8rpe_dd_20uT57zrrLKPVzSvn0ukfZo_VPLefQXHff28qn1jO-zVnap_ba_jYPL7e9a--rDsctNssdm6d